In [15]:
import torch
from torchvision import models, transforms, datasets
from torchsummary import summary
from PIL import Image 

import numpy as np
import matplotlib.pyplot as plt
import time

In [2]:
#******************************
# Define image transformations
#******************************
img_transforms = {
    'train' : transforms.Compose([
        transforms.RandomResizedCrop(size=256),
        transforms.RandomRotation(degrees=15),
        transforms.RandomHorizontalFlip(),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    'valid' : transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ]),
    'test'  : transforms.Compose([
        transforms.Resize(size=256),
        transforms.CenterCrop(size=224),
        transforms.ToTensor(),
        transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
    ])
}


In [11]:
#****************************************************
# Define dataset directory paths and other variables
#****************************************************
#Paths to be used on Google colab
# train_datadir = '/content/drive/MyDrive/Datasets/TransferLearning/train'
# valid_datadir = '/content/drive/MyDrive/Datasets/TransferLearning/valid'
# test_datadir = '/content/drive/MyDrive/Datasets/TransferLearning/test'

#Paths to be used on local machine
train_datadir = '../data/TransferLearning/train'
valid_datadir = '../data/TransferLearning/valid'
test_datadir = '../data/TransferLearning/test'

data = {
    'train' : datasets.ImageFolder(root=train_datadir,transform=img_transforms['train']),
    'valid' : datasets.ImageFolder(root=valid_datadir,transform=img_transforms['valid']),
    'test'  : datasets.ImageFolder(root=test_datadir,transform=img_transforms['test'])
}

train_datalen = len(data['train'])
valid_datalen = len(data['valid'])
test_datalen = len(data['test'])

#Map the indices to the respective class names to see the output of the predictions
idx_to_class = {index: classname for classname, index in data['train'].class_to_idx.items()}

#Load the dataset in batches using the DataLoader
batch_len = 32
train_dataloader = torch.utils.data.DataLoader(data['train'], batch_size= batch_len, shuffle= True)
valid_dataloader = torch.utils.data.DataLoader(data['valid'], batch_size= batch_len, shuffle= True)
test_dataloader  = torch.utils.data.DataLoader(data['test'], batch_size= batch_len, shuffle= True)

print(idx_to_class)

{0: 'bear', 1: 'chimp', 2: 'giraffe', 3: 'gorilla', 4: 'llama', 5: 'ostrich', 6: 'porcupine', 7: 'skunk', 8: 'triceratops', 9: 'zebra'}


In [12]:
#Load the pre-trained model
model_resnet50 = models.resnet50(pretrained=True)

In [13]:
#For transfer Learning we freeze the Feature extraction layer
#Set requires_grad = False to freeze the parameters and 
#the gradients are not computed in the backward() 
for param in model_resnet50.parameters():
    param.requires_grad = False

#Customize the final fully connected layer to fit our dataset
# fully_connected_input = model_resnet50.fc.in_features
model_resnet50.fc = torch.nn.Sequential(
    torch.nn.Linear(2048, 256),
    torch.nn.ReLU(),
    torch.nn.Dropout(0.4),
    torch.nn.Linear(256, 10)
)


#Define the loss function and the optimizer
loss_func = torch.nn.NLLLoss()
optimizer = torch.optim.Adam(model_resnet50.parameters())


In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [1]:
#Define a function for training and validating the custom layers
def train_and_validate(model, loss_function, optimizer, epochs=20):
    training_start_time = time.time()
    history = []
    for epoch in epochs:
        epoch_start_time = time.time()
        print("Epoch {}/{}".format(epoch+1,epochs))

        #Initialize training and validation accuracy and loss for each epoch
        train_acc = 0.00
        train_loss = 0.00
        valid_acc = 0.00
        valid_loss = 0.00

        for i, (inputs,labels) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            #Always clear out gradients in every iteration 
            optimizer.zero_grad()
            #Run a forward pass
            outputs = model(inputs)
            #Compute loss
            loss = loss_function(outputs,labels)
            #Compute gradients using Backpropogation
            loss.backward()
            #Update the parameters such as weights and biases
            optimizer.step()

            #Compute total loss for the batch and add to train_loss
            train_loss += loss.item() * inputs.size(0)

            #To compute accuracy we do predictions find out total correct outputs and calculate mean to obtain the percentage
            _,predictions = torch.max(outputs.data, 1)
            correct_predictions = predictions.eq(labels.data.view_as(predictions))
            #Convert correct_predictions to float and calculate the mean
            accuracy = torch.mean(correct_predictions.type(torch.FloatTensor))
            train_acc += accuracy.item() * inputs.size(0)

        #During validation, gradient tracking is not required
        with torch.no_grad():  
            model.eval()
            for j, (inputs, labels) in valid_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                #Perform a forward pass and run prediction on validation set
                outputs = model(inputs)
                loss = loss_function(outputs, labels)

                #Compute total validation loss
                valid_loss += loss.item() * inputs.size(0)
                
                #Compute validation accuracy
                _, predictions = torch.max(output.data,1)
                correct_predictions = predictions.eq(labels.data.view_as(predictions))
                accuracy = torch.mean(correct_predictions.type(torch.FloatTensor))
                valid_acc += accuracy.item() * inputs.size(0)
        
        avg_train_loss = train_loss/train_datalen
        avg_train_acc = train_acc/train_datalen
        avg_valid_loss = valid_loss/valid_datalen
        avg_valid_acc = valid_acc/valid_datalen

        history.append([avg_train_loss, avg_valid_loss, avg_train_acc, avg_valid_acc])

        epoch_end_time = time.time()
        print("Epoch: {:03f}, Training loss: {:.4f}, Training acc: {:.4f} %,\n\t\tValid loss: {:.4f}, Valid acc: {:.4f} %, Time : {:.4f}s".format(epoch, avg_train_loss, avg_train_acc*100, avg_valid_loss, avg_valid_acc, epoch_start_time - epoch_end_time))

        torch.save(model,'resnet_model_'+str(epoch)+'.pt')

    return model, history

    
